In [1]:
# Headers
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from copy import deepcopy
from skimage.feature import greycomatrix, greycoprops
import cv2  
from numpy.linalg import det
from skimage import util, exposure
from math import sqrt, ceil
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io

In [2]:
# global variables
black_value = np.float64(-1408.5106382978724)
images_filename = 'tr_im.nii'
masks_filename = 'tr_mask.nii'

In [3]:
def get_vals(mask):
    vals = []
    x, y = mask.shape
    for i in range(x):
        for j in range(y):
            if mask[i,j] not in vals:
                vals.append(mask[i][j])

    vals.remove(0)
    return vals

In [4]:
def apply_mask(img, mask, class_id):
    heigh, width = img.shape
    tmp = deepcopy(img)
    lineal_array = np.array(0)
    for x in range(heigh):
        for y in range(width):
            # Set pixel as black color if is not inside this mask
            if mask[x,y] != np.float64(class_id):
                tmp[x,y] = black_value
            # Count pixels inside this mask
            else:
                lineal_array = np.append(lineal_array, tmp[x,y])
    # In tmp is saved the image with mask applied
    # In lineal_array is saved just the mask's pixels
    lineal_array = np.delete(lineal_array, 0)
    return tmp, lineal_to_matrix(lineal_array)

In [5]:
def calculate_media_lineal_arr(array):
    length = array.shape[0]
    media = np.float64(0)
    for x in range(length):
        media += array[x]
    media = media/length
    return media

In [6]:
def lineal_to_matrix(lineal_array):
    sqrt_ = ceil(sqrt(lineal_array.shape[0]))
    media = calculate_media_lineal_arr(lineal_array)
    missing_values = sqrt_**2 - lineal_array.shape[0]
    # Complete square matrix
    for x in range(missing_values):
        lineal_array = np.append(lineal_array, media)
    
    return lineal_array.reshape([sqrt_, sqrt_])

In [7]:
def show_slice_mask(slicei, mask):
    """ Function to display images and mask """
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(slicei.T,  cmap="gray", origin="lower")
    ax[0].set_title('Image')
    ax[1].imshow(mask.T, cmap="gray", origin="lower")
    ax[1].set_title('Mask')
    plt.show()

In [18]:
def glcm_properties(image):
    distancias = [1,2]
    #0 -> Oeste a este, -np.pi/2 -> Norte a sur
    angulos = [0, -np.pi/2]

    # Calculate properties for just one superpixel using the
    # respective mask  
    glcm = greycomatrix(image, 
                        distances = distancias, 
                        angles = angulos,
                        symmetric=True, 
                        normed=True)
                        
    energyResults = greycoprops(glcm, 'energy')
    contrastResults = greycoprops(glcm, 'contrast')
    correlationResults = greycoprops(glcm, 'correlation')
    
    return [np.mean(contrastResults), np.mean(energyResults), np.mean(correlationResults)]

In [28]:
def training(imgs, masks):
    height, width, num_images = imgs.shape
    # Class dictionary to store every property for this class as key
    classes_dict = {}
    # loop over the images
    for x in range(num_images):
        if x+1 % 10 == 0:
            print("{} imagenes procesadas".format(x))
        # Get every class in the current image
        classes = get_vals(masks[:,:,x])
        # loop for every class in the current image 
        for j in classes:
            if not int(j) in classes_dict:
                classes_dict[int(j)] = [[0, 0], [0, 0], [0, 0]]
            # getting the square matrix for the current class
            tmp, matrix_mask = apply_mask(imgs[:,:,x], masks[:,:,x], j)
            # Contrast, Correlation, Energy properties
            props = glcm_properties(matrix_mask.astype(np.uint8))
            # Adding every class's property values
            for z in range(len(props)):
                classes_dict[int(j)][z][0] += props[z]
                classes_dict[int(j)][z][1] += 1

    # Getting average for every class's property
    for id_ in classes_dict.keys():
        for z in range(3):
            classes_dict[id_][z][0] = classes_dict[id_][z][0]/classes_dict[id_][z][1]
            classes_dict[id_][z] = classes_dict[id_][z][0]
    #0: correlation, 1: contrast, 2: energy
    return classes_dict

In [29]:
def main():
    to_show = 3
    imgs = nib.load(images_filename).get_fdata()
    masks = nib.load(masks_filename).get_fdata()
    train = training(imgs, masks)
    print(train)
    """
    #for x in range(3):
    for x in [25, 26, 27]:
        print("Image no ", x)
        classes = get_vals(masks[:,:,x])
        show_slice_mask(imgs[:,:,x], masks[:,:,x])
        for j in classes:
            tmp, matrix_mask = apply_mask(imgs[:,:,x], masks[:,:,x], j)
            print("class ", j)
            print(type(matrix_mask))
            #show_slice_mask(tmp, masks[:,:,x])
            show_slice_mask(tmp, matrix_mask)
            print(glcm_properties(matrix_mask.astype(np.uint8)))
            #create_matrix(tmp)
        #print(get_prop(imgs[:,:,x].astype(np.uint8), masks[:,:,x].astype(np.uint8)))
        #get_every_mask(imgs[:,:,x], masks[:,:,x])
    """

In [30]:
main()

{2: [10596.72999509758, 0.02617827123359886, 0.06471160120920959], 1: [10197.259514566633, 0.014172971679995441, 0.08630341160591291], 3: [11714.680387832675, 0.043996813527231435, 0.05819362993873664]}
